# Ejercicio destilación: Maestro - Estudiante

## Bogdan Kaleb García Rivera 
### MIA-2 



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Definir el modelo maestro y el modelo estudiante
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1) #Aplanar la entrada
        return self.fc(x)

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  #Aplanar la entrada
        return self.fc(x)

In [2]:
# Inicializar los modelos
teacher_model = TeacherModel()
student_model = StudentModel()

# Definir la función de pérdida
criterion_hard = nn.CrossEntropyLoss()
criterion_soft = nn.KLDivLoss(reduction='batchmean')

# Definir el optimizador
optimizer = optim.Adam(student_model.parameters(), lr=0.001)

In [3]:
# Función del entrenamiento del modelo estudiante
def train(student_model, teacher_model, dataloader, alpha, temperature):
    student_model.train()
    for data, target in dataloader:
        optimizer.zero_grad()

        # Generar etiquetas suaves con el modelo maestro
        with torch.no_grad():
            teacher_output = teacher_model(data)
            soft_target = torch.softmax(teacher_output / temperature, dim=1)

        # Salidas del modelo estudiante
        student_output = student_model(data)
        hard_loss = criterion_hard(student_output, target)
        soft_loss = criterion_soft(torch.log_softmax(student_output / temperature, dim=1), soft_target)

        # Pérdida total
        loss = alpha * hard_loss + (1 - alpha) * soft_loss

        # Retropropagación y optimización
        loss.backward()
        optimizer.step()

# Convertidor a tipo de dato apto para la red
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)), # Media y mediana de acuerdo a un estándar de un artículo
    transforms.Lambda(lambda x: torch.flatten(x))  # Aplana a 784
])

In [4]:
def train_teacher(teacher_model, dataloader, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(teacher_model.parameters(), lr=0.001)
    
    teacher_model.train()
    for epoch in range(epochs):
        for data, target in dataloader:
            optimizer.zero_grad()
            output = teacher_model(data)  
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Épocas del maestro {epoch+1}, Loss: {loss.item():.4f}")

In [5]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in dataloader:
            output = model(data)  
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [6]:
# Descargar y cargar el conjunto de datos MNIST de entrenamiento
train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)
# Carga de datos de entrenamiento 
train_loader = DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True
)

# Datos de prueba
test_dataset = datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Entrenar al maestro
train_teacher(teacher_model, train_loader, epochs=5)

# Evaluación del modelo antes de la destilación
teacher_eval = evaluate(teacher_model, test_loader)
student_eval = evaluate(student_model, test_loader)
print(f"Precisión del maestro antes de la destilación: {teacher_eval:.2f}%")
print(f"Precisión del estudiante antes de la destilación: {student_eval:.2f}%")

# Entrenar el modelo del estudiante
train(student_model, teacher_model, train_loader, alpha=0.5, temperature=2.0)

# Evaluación del estudiante después de la destilación 
student_acc_after = evaluate(student_model, test_loader)
print(f"Precisión del estudiante después de la destilación: {student_acc_after:.2f}%")


Épocas del maestro 1, Loss: 0.3729
Épocas del maestro 2, Loss: 0.5040
Épocas del maestro 3, Loss: 0.1402
Épocas del maestro 4, Loss: 0.1446
Épocas del maestro 5, Loss: 0.3956
Precisión del maestro antes de la destilación: 92.01%
Precisión del estudiante antes de la destilación: 16.27%
Precisión del estudiante después de la destilación: 91.79%
